In [1]:
import pandas as pd
import pickle
import my_func
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.model_selection import KFold 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from datetime import datetime, timedelta
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import seaborn as sns
import matplotlib.pyplot as plt
import time
from sklearn.metrics import confusion_matrix
import shap
from hyperopt import hp, fmin, tpe

sns.set()

fe_name_tr = 'churn_train_model_fe.pickle'
b_num_name_tr ='churn_train_model_b_num.pickle'
dpi_name_tr ='churn_train_model_dpi.pickle'

fe_name_te = 'churn_test_model_fe.pickle'
b_num_name_te ='churn_test_model_b_num.pickle'
dpi_name_te ='churn_test_model_dpi.pickle'

file_path = r'D:\BDL-Final\data-from-BDL' + '\\'

# settings for dataframe
pd.set_option('display.max_columns', None)

In [19]:
# Load dataframes
df_train_fe = pd.read_pickle(file_path+fe_name_tr)
df_train_dpi = pd.read_pickle(file_path+dpi_name_tr)
df_train_b_num = pd.read_pickle(file_path+b_num_name_tr)

df_test_fe = pd.read_pickle(file_path+fe_name_te)
df_test_dpi = pd.read_pickle(file_path+dpi_name_te)
df_test_b_num = pd.read_pickle(file_path+b_num_name_te)

In [3]:
len(df_test_fe['abon_id'].unique())

150000

In [ ]:



# Unique values of 'Application'
unique_abon_ids = df_train_dpi['abon_id'].unique()
unique_values_train = df_train_dpi['Application'].unique()
unique_values_test = df_test_dpi['Application'].unique()
unique_applications = np.unique(np.concatenate((unique_values_train, unique_values_test)))
#unique_applications = np.array([14, 23])

# Create the df_t dataframe with 'abon_id' column
df_t = pd.DataFrame({'abon_id': unique_abon_ids})

# Pre-filter df_train_dpi for each application
filtered_dfs = {}
for app in unique_applications:
    filtered_dfs[app] = df_train_dpi[df_train_dpi['Application'] == app]

# Initialize dictionaries to store column values
column_values = {}
for app in unique_applications:
    col_names = [f"{app}_SUM_of_Duration_sec", f"{app}_SUM_of_Volume_kb", f"{app}_SUM_of_Count_events", f"{app}_MAX_of_day_cnt"]
    column_values[col_names[0]] = []
    column_values[col_names[1]] = []
    column_values[col_names[2]] = []
    column_values[col_names[3]] = []

# Progress update interval in seconds
progress_interval = 3 * 60  # 3 minutes

# Iterate over each row in df_t
total_rows = len(df_t)
start_time = time.time()
last_progress_time = start_time
for row_index, row in df_t.iterrows():
    abon_id = row['abon_id']
    
    # Iterate over each application
    for app_index, app in enumerate(unique_applications):
        col_names = [f"{app}_SUM_of_Duration_sec", f"{app}_SUM_of_Volume_kb", f"{app}_SUM_of_Count_events", f"{app}_MAX_of_day_cnt"]
        filtered_df = filtered_dfs[app]
        
        # Find the values for the current abon_id and application
        values = filtered_df.loc[filtered_df['abon_id'] == abon_id, ['SUM_of_Duration_sec', 'SUM_of_Volume_kb', 'SUM_of_Count_events', 'MAX_of_day_cnt']].values
        
        if len(values) > 0:
            column_values[col_names[0]].append(values[0][0])
            column_values[col_names[1]].append(values[0][1])
            column_values[col_names[2]].append(values[0][2])
            column_values[col_names[3]].append(values[0][3])
        else:
            column_values[col_names[0]].append(None)
            column_values[col_names[1]].append(None)
            column_values[col_names[2]].append(None)
            column_values[col_names[3]].append(None)
    
    # Print progress every 3 minutes
    current_time = time.time()
    elapsed_time = current_time - last_progress_time
    if elapsed_time >= progress_interval:
        progress = (row_index + 1) / total_rows * 100
        print(f"Progress: {progress:.2f}%")
        last_progress_time = current_time

# Add the values as new columns to df_t
for col_name, values in column_values.items():
    df_t[col_name] = values

# Print the resulting df_t dataframe
#print(df_t)


In [6]:
df_t.to_pickle('df_train_dpi_t_v2.pickle')


In [3]:

df_new = pd.DataFrame(df_train_b_num['abon_id'].unique(), columns=['abon_id'])

In [6]:
df_train_b_num

,bnum,call_cnt_out,call_cnt_in,call_dur_out,call_dur_in,cnt_sms_out,cnt_sms_in,abon_id
0,b'dsns ukr',0.0,0.0,0.0,0.0,0.0,3.590290,1545052.0
1,b'polgazzbut',0.0,0.0,0.0,0.0,0.0,4.786566,1545052.0
2,b'cyberpolic',0.0,0.0,0.0,0.0,0.0,2.921812,1545235.0
3,b'dsns ukr',0.0,0.0,0.0,0.0,0.0,3.590290,1545235.0
4,b'e-health',0.0,0.0,0.0,0.0,0.0,2.206949,1545235.0
...,...,...,...,...,...,...,...,...
671243,b'whatsapp',0.0,0.0,0.0,0.0,0.0,1.480453,130754629.0
671244,b'apple',0.0,0.0,0.0,0.0,0.0,1.480453,130754935.0
671245,b'icu',0.0,0.0,0.0,0.0,0.0,5.324077,130754935.0
671246,b'novaposhta',0.0,0.0,0.0,0.0,0.0,3.590290,130754935.0


In [7]:

# List of bnum values to include in the transformed DataFrame
bnum_list = ['111', '3700', '380800210800', '1545', '112', '102', '103', '380505022250', '380502907290']

# Remove the 'b' prefix and single quotes from the 'bnum' column
df_train_b_num['bnum'] = df_train_b_num['bnum'].str[2:-1]

# Create a new DataFrame with unique abon_id
df_transformed = pd.DataFrame(df_train_b_num['abon_id'].unique(), columns=['abon_id'])

# Create a new column for each value in bnum_list
for bnum_value in bnum_list:
    df_transformed[bnum_value] = 0

# Iterate over the rows in the original DataFrame
for i in df_train_b_num.index:
    # Get the current row's 'abon_id', 'bnum', and 'call_cnt_out' values
    abon_id = df_train_b_num.loc[i, 'abon_id']
    bnum = df_train_b_num.loc[i, 'bnum']
    call_cnt_out = df_train_b_num.loc[i, 'call_cnt_out']
    
    # If 'call_cnt_out' is not null and 'bnum' is in the transformed DataFrame's columns
    if pd.notnull(call_cnt_out) and bnum in df_transformed.columns:
        # Find the row in the transformed DataFrame with the same 'abon_id'
        row_index = df_transformed[df_transformed['abon_id'] == abon_id].index[0]
        
        # Update the 'bnum' column in that row with the 'call_cnt_out' value
        df_transformed.loc[row_index, bnum] = call_cnt_out

df_transformed


,abon_id,111,3700,380800210800,1545,112,102,103,380505022250,380502907290
0,1545052.0,0,0,0,0,0,0,0,0,0
1,1545235.0,0,0,0,0,0,0,0,0,0
2,1549591.0,0,0,0,0,0,0,0,0,0
3,1558772.0,0,0,0,0,0,0,0,0,0
4,1558921.0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
138697,130753784.0,0,0,0,0,0,0,0,0,0
138698,130753819.0,0,0,0,0,0,0,0,0,0
138699,130754333.0,0,0,0,0,0,0,0,0,0
138700,130754629.0,0,0,0,0,0,0,0,0,0


In [4]:
df_transformed.to_excel('num_t.xlsx')

In [8]:
df_transformed['111'].sum()

0

In [9]:
# List of bnum values to include in the transformed DataFrame
bnum_list = ['111', '3700', '380800210800', '1545', '112', '102', '103', '380505022250', '380502907290']

# Filter the DataFrame to include only the rows with bnum in bnum_list
df_filtered = df_train_b_num[df_train_b_num['bnum'].isin(bnum_list)]

# Create a pivot table with 'abon_id' as index, 'bnum' as columns, and 'call_cnt_out' as values
# Fill missing values with 0
df_transformed = df_filtered.pivot_table(index='abon_id', columns='bnum', values='call_cnt_out', fill_value=0)

# Reset the index to make 'abon_id' a column again
df_transformed.reset_index(inplace=True)

df_transformed


bnum,abon_id


In [17]:
#df_train_b_num['bnum'] = df_train_b_num['bnum'].str[2:-1]


#df_train_b_num.pivot_table(values='call_cnt_out',index='abon_id',aggfunc=np.sum)

<class 'pandas.core.series.Series'>
RangeIndex: 671248 entries, 0 to 671247
Series name: bnum
Non-Null Count   Dtype 
--------------   ----- 
671248 non-null  object
dtypes: object(1)
memory usage: 5.1+ MB


In [30]:
bnum_list = ["b'111'", "b'3700'", "b'380800210800'", "b'1545'", "b'112'", "b'102'", "b'103'", "b'380505022250'", "b'380502907290'"]
df_train_b_num['bnum'] = df_train_b_num['bnum'].astype(str)

# Filter the DataFrame based on the 'bnum' column
df_filtered = df_train_b_num[df_train_b_num['bnum'].str.contains('|'.join(bnum_list))]



# # Pivot the filtered DataFrame
pivot_table = df_filtered.pivot_table(values='call_cnt_out', index='abon_id', columns='bnum', aggfunc=np.sum)

# # Fill NaN values with 0
# pivot_table = pivot_table.fillna(0)

pivot_table['b\'103\'']


bnum,b'102',b'103',b'111',b'112',b'1545',b'3700',b'380502907290',b'380505022250',b'380800210800'
abon_id,,,,,,,,,
1560464.0,NaN,NaN,NaN,NaN,NaN,2.921812,NaN,NaN,NaN
1610598.0,NaN,NaN,NaN,NaN,NaN,NaN,2.206949,NaN,NaN
1610707.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.921812
1612053.0,NaN,NaN,NaN,NaN,NaN,2.206949,NaN,NaN,NaN
1637193.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.480453
...,...,...,...,...,...,...,...,...,...
130748999.0,NaN,NaN,1.480453,NaN,NaN,NaN,NaN,NaN,NaN
130749792.0,NaN,NaN,3.590290,NaN,NaN,NaN,NaN,NaN,NaN
130753073.0,NaN,NaN,2.206949,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
pivot_table['b\'103\''].sum()


1101.872683656741